In [1]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=2
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' + '1'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
res = requests.get(url,headers=header)
soup = BeautifulSoup(res.text,'html.parser')

In [3]:
len(res.text) 

292070

In [4]:
res.text

'<!doctype html>\n<html lang="ko">\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="imagetoolbar" content="no">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <title>유튜브 랭킹 1 페이지</title>\n    <link rel="stylesheet" href="/css/default.css?ver=2022022">\n<link rel="stylesheet" href="/css/bootstrap.css?ver=2022022">\n<link rel="stylesheet" href="/css/nifty.min.css?ver=2022022">\n<link rel="stylesheet" href="/css/premium/icon-sets/icons/line-icons/premium-line-icons.min.css?ver=2022022" >\n<link rel="stylesheet" href="/css/themes/type-c/theme-well-red.css?ver=2022022">\n<link rel="stylesheet" href="https://youtube-rank.com/board/theme/youtube/skin/outlogin/youtuberank/style.css?ver=2022022">\n<link rel="stylesheet" href="https://youtube-rank.com/board/skin/board/sns_channel/list.skin.css?2022022" />\n<link rel="stylesheet" href="https://unpkg.com/aos@2.3.1/dist/aos.css" />\n    <link rel="apple-touch-icon" sizes="180x180" href="/img/apple-touch-icon.png">\n    <link 

In [5]:
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe') # chromedriver 위치
driver.get(url)

In [6]:
soup = BeautifulSoup(driver.page_source)
trs = soup.select('.aos-init')
len(trs)

100

In [7]:
len(driver.page_source), len(res.text) #request로 가져오는것은 제한이 걸려있을수 있음

(296571, 292070)

In [8]:
tr = trs[0]

In [9]:
# 순위
rank = tr.select_one('.rank').get_text().strip()
rank

'1'

In [10]:
# 분야
category = tr.select_one('.category').get_text().strip()[1:-1]
category

'음악/댄스/가수'

In [11]:
#list-skin > form:nth-child(5) > table > tbody > tr:nth-child(1) > td.subject > h1 > a
# 채널명
channel = tr.select_one('.subject > h1 > a').get_text().strip()
channel

'BLACKPINK'

In [12]:
#list-skin > form:nth-child(5) > table > tbody > tr:nth-child(1) > td.subscriber_cnt
#구독자순(정수로)
follow = tr.select_one('td.subscriber_cnt').get_text().strip()
follow = int(follow[:-1])*10000
follow


87300000

In [13]:
# View순(정수로)
view_str = tr.select_one('td.view_cnt').get_text().strip()
view_str

'305억3350만'

In [14]:
# View순(정수로)
view = tr.select_one('td.view_cnt').get_text().strip()
view

'305억3350만'

In [15]:
# Video순(정수로)
video = tr.select_one('td.video_cnt').get_text().strip()
video = int(video[:-1])
video

507

In [16]:
# 숫자로 바꿔주는 함수
def convert(s):
    s = s.replace('억','').replace('개','').replace(',','').replace('만','0000')
    return int(s)

In [17]:
lines = []
for tr in trs:
    rank = int(tr.select_one('.aos-init >.rank').get_text().strip())
    category = tr.select_one('.category').get_text().strip()
    channel = tr.select_one('.subject > h1 > a').get_text().strip()
    follow = tr.select_one('td.subscriber_cnt').get_text().strip()
    view = tr.select_one('td.view_cnt').get_text().strip()
    video = tr.select_one('td.video_cnt').get_text().strip()
    lines.append({'순위':rank, '분야':category, '채널명':channel, '구독자':follow, '조회수':view, 'video':video})

In [18]:
df = pd.DataFrame(lines)
df

,순위,분야,채널명,구독자,조회수,video
0,1,[음악/댄스/가수],BLACKPINK,8730만,305억3350만,507개
1,2,[음악/댄스/가수],BANGTANTV,7450만,201억4300만,"2,188개"
2,3,[음악/댄스/가수],HYBE LABELS,7030만,273억2316만,"1,158개"
3,4,[음악/댄스/가수],SMTOWN,3160만,267억9481만,"4,110개"
4,5,[음악/댄스/가수],JYP Entertainment,2680만,192억7507만,"1,668개"
...,...,...,...,...,...,...
95,95,[키즈/어린이],뽀로로(Pororo),486만,68억5315만,"4,160개"
96,97,[애완/반려동물],SBS TV동물농장x애니멀봐,483만,46억3810만,"4,164개"
97,98,[키즈/어린이],CoCosToy 꼬꼬스토이,477만,29억8364만,686개
98,99,[음식/요리/레시피],Cooking tree 쿠킹트리,476만,4억8758만,"1,450개"


- 두번째 페이지

In [19]:
# https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=2
import time
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' + str(2)
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe') # chromedriver 위치
driver.get(url)
time.sleep(2) # 페이지 이동할때 
soup = BeautifulSoup(driver.page_source,'html.parser')
trs = soup.select('.aos-init')
len(trs)

100

In [20]:
lines = []
for page in range(1,11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' + str(page)
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    trs = soup.select('.aos-init')

    for tr in trs:
        rank =int(tr.select_one('.info_rank').get_text().strip())
        category=tr.select_one('.category').get_text().strip()[1:-1]
        title = tr.select_one('h1 > a').get_text().strip()
        subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
        view = convert(tr.select_one('.view_cnt').get_text().strip())
        video = convert(tr.select_one('.video_cnt').get_text().strip())
        lines.append({'순위':rank,'카테고리':category,'채널명':title,'구독자수':subscriber,
                    '조회수':view,'비디오':video})
driver.close()

In [21]:
df = pd.DataFrame(lines)
df.shape

(1000, 6)

In [22]:
df.tail()

,순위,카테고리,채널명,구독자수,조회수,비디오
995,991,회사/오피셜,LG Uplus,730000,1379980000,2473
996,998,BJ/인물/연예인,케이,730000,572670000,3304
997,1001,미분류,백크 Baekk,720000,457320000,1106
998,999,음악/댄스/가수,Crush,720000,395460000,131
999,997,BJ/인물/연예인,bobaepapa - 보배아빠,720000,348440000,393


In [23]:
df.to_csv('data/20230425_YouTube랭킹.csv', index=False)